In [1]:
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os
from osgeo import gdal
import glob

loading dot env...


In [2]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
    
    
)

bucket = constants.AWS_S3_BUCKET



In [3]:
#list contents of bucket 
response = s3_client.list_objects_v2(Bucket=bucket)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No objects found in the bucket.")

2017_wildfire_imagery/C10683/wvge_c10683_c1r1_2017_bcalb_500mm_fcir_enh.sid
2017_wildfire_imagery/C10683/wvge_c10683_c1r1_2017_bcalb_500mm_rgb_enh.sid
STAC_DEM/
STAC_DEM/Catalog/DEM_Test.json
STAC_DEM/Catalog/bc_092o020_xli1m_utm10_2018_cog
STAC_DEM/Data/
STAC_DEM/Data/bc_092o018_xli1m_utm10_2018_2_cog.tif
STAC_DEM/Data/bc_092o018_xli1m_utm10_2018_cog.tif
STAC_DEM/Data/bc_092o019_xli1m_utm10_2018_2_cog.tif
STAC_DEM/Data/bc_092o019_xli1m_utm10_2018_cog.tif
STAC_DEM/Data/bc_092o020_xli1m_utm10_2018_2_cog.tif
STAC_DEM/Data/bc_092o020_xli1m_utm10_2018_cog.tif
STAC_DEM/Data/bc_092o028_xli1m_utm10_2018_cog.tif
STAC_DEM/Data/bc_092o029_xli1m_utm10_2019_cog.tif
STAC_DEM/Data/bc_092o030_xli1m_utm10_2018_cog.tif
catalog.json
how2find_imagery_120114.pdf
mgtest/bc_092i045.tif


In [16]:
#Create Folder Like Structure(empty objects) for STAC Lidar base area and point clouds, if they do no exist already  .laz files will go into point clouds object
base_obj="STAC_DEM/"
pc_obj="STAC_DEM/Data/"
ct_obj="STAC_DEM/Catalogs/"
obj_list=[base_obj,pc_obj, ct_obj]

print(obj_list)
for item in obj_list:
    response = s3_client.list_objects_v2(Bucket=bucket, Prefix=item)
    if 'Contents' in response:
        for obj in response['Contents']:
            if obj['Key'] == item:
                print(f"The object {item}' in bucket '{bucket}' exists.")
                break
        else:
            print(f"The object {item}' in bucket '{bucket}' dose not exist, adding object")
            s3_client.put_object(Bucket=bucket, Key=item)


s3_client.put_object(Bucket=bucket, Key=pc_obj)

['STAC_DEM/', 'STAC_DEM/Data/', 'STAC_DEM/Catalogs/']
The object STAC_DEM/' in bucket 'cloudgistest' exists.
The object STAC_DEM/Data/' in bucket 'cloudgistest' exists.


{'ResponseMetadata': {'RequestId': '8e22ee10:18fb31a035c:d2d2:3805',
  'HostId': '59270a9007ad42687d56a72a68420adec4ebd7245ff960c4a93f9baf5ad3e351',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Thu, 30 May 2024 14:46:45 GMT',
   'server': 'ViPR/1.0',
   'x-amz-request-id': '8e22ee10:18fb31a035c:d2d2:3805',
   'x-amz-id-2': '59270a9007ad42687d56a72a68420adec4ebd7245ff960c4a93f9baf5ad3e351',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'last-modified': 'Thu, 30 May 2024 14:46:45 GMT',
   'x-emc-mtime': '1717080405112',
   'x-emc-previous-object-size': '0',
   'x-amz-version-id': '1717080405112',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'VersionId': '1717080405112'}

In [9]:
# specify the folder path (in the current working directory) where objects are/will be
# if the folder doesn't exist, create it
save_path = (r'/home/cfolkers/STAC_LiDAR/DEM')
if not os.path.exists(save_path):
    os.makedirs(save_path)


In [10]:
#create COGSs
DEM_list=os.listdir(save_path)

options = ['COMPRESS=LZW', 'NUM_THREADS=ALL_CPUS', 'BIGTIFF=YES']


for f in DEM_list:
    f=os.path.join(save_path,f)
    splits=f.split('.')
    output_cog=f"{splits[0]}_cog.{splits[1]}"
    input_dataset = gdal.Open(f)
    gdal.Translate(output_cog, input_dataset, format='COG', creationOptions=options)
    input_dataset = None  


In [12]:
# put .laz files into your object storage "STAC_LiDAR/PointClouds/"

DEM_list= glob.glob(fr"{save_path}/*_cog.*")
for obj in DEM_list:
    obj_load=os.path.join(save_path,obj)
    obj_key=pc_obj+os.path.basename(obj)
    # s3_client.upload_file(obj_load, Bucket=bucket, Key=obj_key)
    
print(f"Objects uploaded to folder '{pc_obj}' in bucket '{bucket}'.")


KeyboardInterrupt: 

In [17]:
#list .laz objects in bucket
# object_key="STAC_LiDAR/PointClouds/"
laz_objects=[]

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=pc_obj, StartAfter=pc_obj)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        laz_objects.append(obj['Key'])
        print(obj['Key'])
        print(f"Object Size {obj['Size']}")
else:
    print("No objects found in the bucket.")

STAC_DEM/Data/bc_092o018_xli1m_utm10_2018_2_cog.tif
Object Size 27945230
STAC_DEM/Data/bc_092o018_xli1m_utm10_2018_cog.tif
Object Size 131166704
STAC_DEM/Data/bc_092o019_xli1m_utm10_2018_2_cog.tif
Object Size 281923918
STAC_DEM/Data/bc_092o019_xli1m_utm10_2018_cog.tif
Object Size 256676853
STAC_DEM/Data/bc_092o020_xli1m_utm10_2018_2_cog.tif
Object Size 19758214
STAC_DEM/Data/bc_092o020_xli1m_utm10_2018_cog.tif
Object Size 370620362
STAC_DEM/Data/bc_092o028_xli1m_utm10_2018_cog.tif
Object Size 53380148
STAC_DEM/Data/bc_092o029_xli1m_utm10_2019_cog.tif
Object Size 20150088
STAC_DEM/Data/bc_092o030_xli1m_utm10_2018_cog.tif
Object Size 80279565


In [6]:
### Run only to delete all objects 
pc_obj="STAC_LiDAR/DEM/"
response = s3_client.list_objects_v2(Bucket=bucket, Prefix=pc_obj, StartAfter=pc_obj)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        print(obj['Key'])
        s3_client.delete_object(Bucket=bucket, Key=obj['Key']
    )

STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o018_xli1m_utm10_2018_2_cog.tif
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o018_xli1m_utm10_2018_cog.tif
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o019_xli1m_utm10_2018_2_cog.tif
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o019_xli1m_utm10_2018_cog.tif
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o020_xli1m_utm10_2018_2_cog.tif
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o020_xli1m_utm10_2018_cog.tif
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o028_xli1m_utm10_2018_cog.tif
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o029_xli1m_utm10_2019_cog.tif
STAC_LiDAR/DEM//home/cfolkers/STAC_LiDAR/DEM/bc_092o030_xli1m_utm10_2018_cog.tif
